In [1]:
from requests import get
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd

In [2]:
# YEAR_RANGE is (inclusive, exclusive) = (first year, current_year)
FIRST_YEAR = 2000
CURRENT_YEAR = 2021
URL = 'http://www.drafthistory.com/index.php/rounds/round_1'

In [3]:
# Takes 5 seconds to run
response = get(URL)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find_all('table')
df = pd.read_html(str(table))[0]

In [4]:
df

,0,1,2,3,4,5,6,7
0,Round 1 Picks,Round 1 Picks,Round 1 Picks,Round 1 Picks,Round 1 Picks,Round 1 Picks,Round 1 Picks,Round 1 Picks
1,Year,Round,Pick,Player,Name,Team,Position,College
2,2020,1,1,1,Joe Burrow,Bengals,QB,Louisiana State
3,NaN,1,2,2,Chase Young,Redskins,DE,Ohio State
4,NaN,1,3,3,Jeff Okudah,Lions,DB,Ohio State
...,...,...,...,...,...,...,...,...
1990,NaN,1,5,5,Jim Lawrence,Cardinals,B,Texas Christian
1991,NaN,1,6,6,Joe Stydahar,Bears,T,West Virginia
1992,NaN,1,7,7,Russ Letlow,Packers,G,San Francisco
1993,NaN,1,8,8,Sid Wagner,Lions,G,Michigan State


In [5]:
# Drop Top Label Rows And Convert To Column Names
df.drop(0, inplace=True)
df.rename(columns = {k:df.loc[1,k] for k in df.columns}, inplace=True)
df.drop(1, inplace=True)

# Drop Unnecessary Columnsd
df.drop(columns=['Player', 'College'], inplace=True)

# Fill In Years For Null
for row in df.index:
    if pd.isnull(df.loc[row, 'Year']):
        df.loc[row, 'Year'] = df.loc[row-1, 'Year']
        
# Keep Certain Years
df = df[df['Year'].isin([str(i) for i in range(FIRST_YEAR, CURRENT_YEAR)])]

In [6]:
df

,Year,Round,Pick,Name,Team,Position
2,2020,1,1,Joe Burrow,Bengals,QB
3,2020,1,2,Chase Young,Redskins,DE
4,2020,1,3,Jeff Okudah,Lions,DB
5,2020,1,4,Andrew Thomas,Giants,T
6,2020,1,5,Tua Tagovailoa,Dolphins,QB
...,...,...,...,...,...,...
665,2000,1,27,Anthony Becht,Jets,TE
666,2000,1,28,Rob Morris,Colts,LB
667,2000,1,29,R. Jay Soward,Jaguars,WR
668,2000,1,30,Keith Bulluck,Titans,LB


In [7]:
df.to_csv(str(FIRST_YEAR) + '-' + str(CURRENT_YEAR-1) + ' 1st Round Draft Picks.csv', encoding='utf-8-sig', index=False)